In [ ]:
!pip install --upgrade keras
from google.colab import drive
drive.mount('/content/drive')

In [0]:
DIC = './'

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf
import os
import numpy as np
import cv2 as cv
from sklearn.utils import shuffle

tf.enable_eager_execution()

In [0]:
path = f'{DIC}English/Fnt/'

img_rows, img_cols, img_channels = [40, 40, 1]
nb_classes = 62

nb_conv = 3
nb_pool = 2

In [0]:
def load_data(dirs, img_rows, img_cols, img_channels):
    X = []
    y = []
    label = 0
    print('loading images ...')
    for d in dirs:
        dir_path = os.path.join(path, d)
        image_list = os.listdir(dir_path)
        for im in image_list:
            img_path = os.path.join(dir_path, im)
            if img_channels == 1:
                img = cv.imread(img_path, 0)
            else:
                img = cv.imread(img_path)
            img = cv.resize(img, (img_rows, img_cols))
            X.append(img)
            y.append(label)
        label = label + 1
    print('Complete!')
    return X, y


def prepare_data_for_training(X, y, img_rows, img_cols, img_channels):
    X = np.asarray(X, dtype='float32')
    y = np.asarray(y)
    if img_channels == 1:
        X = X.reshape(X.shape[0], img_channels, img_rows, img_cols)
    else:
        X = np.transpose(X, (0, 3, 1, 2))

    X, y = shuffle(X, y, random_state=7)  # randomly shuffles the data

    y = tf.keras.utils.to_categorical(y, nb_classes)  # converts integer labels to one-hot vecor
    return X, y


dirs = sorted(os.listdir(path))

X, y = load_data(dirs, img_rows, img_cols, img_channels)

X, y = prepare_data_for_training(X, y, img_rows, img_cols, img_channels=1)

loading images ...
Complete!


In [0]:
X = np.transpose(X, (0, 2, 3, 1))

In [0]:
def create_model():
    print("Creating model...")
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.BatchNormalization(input_shape=(img_rows, img_cols, img_channels)))

    model.add(tf.keras.layers.Convolution2D(32, (nb_conv, nb_conv), padding='same', strides=(2, 2)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Convolution2D(32, (nb_conv, nb_conv), padding='same'))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.MaxPooling2D(pool_size=(nb_pool, nb_pool), strides=(1, 1)))

    model.add(tf.keras.layers.Convolution2D(48, (nb_conv, nb_conv), padding='same', strides=(2, 2)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Convolution2D(48, (nb_conv, nb_conv), padding='same'))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.MaxPooling2D(pool_size=(nb_pool, nb_pool), strides=(1, 1)))

    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(256, kernel_initializer='he_normal'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))

    model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(nb_classes))
    model.add(tf.keras.layers.Activation('softmax'))

    sgd = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-7, nesterov=True)

    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    model.summary()

    print("Done.")
    return model


model = create_model()

Creating model...
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_17 (Batc (None, 40, 40, 1)         4         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 20, 20, 32)        320       
_________________________________________________________________
activation_15 (Activation)   (None, 20, 20, 32)        0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 20, 20, 32)        128       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 20, 20, 32)        9248      
_________________________________________________________________
activation_16 (Activation)   (None, 20, 20, 32)        0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 

In [0]:
filepath = f"{DIC}sgd.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.fit(X, y, callbacks=[checkpoint], validation_split=0.20, epochs=50, shuffle=True, batch_size=64, verbose=1)